In [1]:
from selenium.webdriver import Firefox
from selenium.webdriver.common.by import By
import sys, importlib

In [2]:
# importlib.reload(sys.modules['pages.elements'])
# importlib.reload(sys.modules['pages.pages'])
from pages.pages import *
from pages.elements import *

In [6]:
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
with open('credenciais.txt') as f:
    login = f.readline()
    senha = f.readline()

browser = Firefox()

pagina = PageLogin(
    browser,
    'https://guardiaov4.seplag.ce.gov.br/auth'
)
pagina.open()

pagina.login.logar(login, senha)
# sleep(4)

pagina = PageGuardiao(browser)
pagina.planejamento.acessarSpg()
# sleep(5)

pagina = PageSPG(browser)
pagina.menu_superior.clicar_sister()
sleep(2)

pagina.avisos.fechar_avisos()
sleep(2)

pagina.menu_lateral.clicar_fatura()
pagina.menu_lateral.clicar_controle_de_fatura()

pagina = PageFatura(browser)

In [ ]:
pagina = PageFatura(browser)
pagina.terceirizados.carregar_funcionarios()
f = pagina.terceirizados.funcionarios[0]

In [ ]:
num_popups = len(pagina.find_elements((By.CSS_SELECTOR, 'button.close')))
for _ in range(num_popups):
    for e in pagina.find_elements((By.CSS_SELECTOR, 'button.close')):
        try: 
            e.click()
        except:
            pass
    sleep(0.1)
